# 注意点：保存名、important_features

# 1 数据导入

○ 缺失值：model(1|0), bodyType(4506|1413), fuelType(8680|2893), gearbox(5981|1910), notRepairedDamage(24324|8031) 

○ 训练时无用特征：SaleID, name, seller, offerType

○ 可直接使用的特征(17)：power, kilometer, v系列

○ 需要转换再使用的特征(8)：regDate, model, brand, bodyType, fuelType, gearBox, notRepairedDamage, regionCode, creatDate

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

path = '/home/agwave/Data/used-car/'
Train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
Test_data = pd.read_csv(path+'used_car_testA_20200313.csv', sep=' ')

In [2]:
# 初步筛选
X_data  = Train_data.drop(['name', 'seller', 'offerType'], axis=1)

X_test = Test_data.drop(['name', 'seller', 'offerType'], axis=1)

In [3]:
X_data.head()

,SaleID,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,19960908,109.0,10,0.0,0.0,1.0,193,15.0,0.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,20120103,110.0,5,1.0,0.0,0.0,68,5.0,0.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [4]:
# notRepairedDamage 类型处理
import numpy as np

X_data['notRepairedDamage'].replace('-', np.nan, inplace=True)
X_data['notRepairedDamage'] = X_data['notRepairedDamage'].astype('float64')

X_test['notRepairedDamage'].replace('-', np.nan, inplace=True)
X_test['notRepairedDamage'] = X_test['notRepairedDamage'].astype('float64')

# 2. 特征工程

## 2.1 useTime

In [5]:
# 日期信息处理，生成usedDay, usedWeek, usedMonth, usedYear
import datetime

data_date = X_data[['regDate', 'creatDate']]
test_date = X_test[['regDate', 'creatDate']]


def get_year_month_day_by_intdate(int_date):
    # 实现通过一个数值类型的日期转换出年月日
    try:
        day = int_date % 100
        month = int_date % 10000 // 100
        year = int_date // 10000
        if month == 0:
            month += 1
    except Exception as e:
        print(e)
        if year is None:
            year = 0
        if month is None:
            month = 0
        if day is None:
            day = 0
    return year, month, day

def get_used_time_by_regDate_and_creatDate(regDate, creatDate):
    r_y, r_m, r_d = get_year_month_day_by_intdate(regDate)
    c_y, c_m, c_d = get_year_month_day_by_intdate(creatDate)
    r = datetime.datetime(r_y, r_m, r_d)
    c = datetime.datetime(c_y, c_m, c_d)
    return (r-c).days

def creatMonthProcess(month):
    if month != 3 and month != 4:
        month = 1
    return month

data_date.loc[:, ('usedDay')] = list(map(lambda x, y: get_used_time_by_regDate_and_creatDate(x, y), data_date.loc[:, ('creatDate')], data_date.loc[:, ('regDate')]))
test_date.loc[:, ('usedDay')] = list(map(lambda x, y: get_used_time_by_regDate_and_creatDate(x, y), test_date.loc[:, ('creatDate')], test_date.loc[:, ('regDate')]))
data_date.loc[:, ('regYear')] = data_date['regDate'].apply(lambda x: get_year_month_day_by_intdate(x)[0])
test_date.loc[:, ('regYear')] = test_date['regDate'].apply(lambda x: get_year_month_day_by_intdate(x)[0])
data_date.loc[:, ('regMonth')] = data_date['regDate'].apply(lambda x: get_year_month_day_by_intdate(x)[1])
test_date.loc[:, ('regMonth')] = test_date['regDate'].apply(lambda x: get_year_month_day_by_intdate(x)[1])
data_date.loc[:, ('creatMonth')] = data_date['creatDate'].apply(lambda x: creatMonthProcess(get_year_month_day_by_intdate(x)[1]))
test_date.loc[:, ('creatMonth')] = test_date['creatDate'].apply(lambda x: creatMonthProcess(get_year_month_day_by_intdate(x)[1]))

X_data['usedDay'] = data_date['usedDay']
X_data['regYear'] = data_date['regYear']
X_data['regMonth'] = data_date['regMonth']
X_data['creatMonth'] = data_date['creatMonth']
X_data['usedYear'] = data_date['usedDay'].apply(lambda x: x//356)
X_data['usedMonth'] = data_date['usedDay'].apply(lambda x: x//30)
X_data['usedWeek'] = data_date['usedDay'].apply(lambda x: x//7)

X_test['usedDay'] = test_date['usedDay']
X_test['regYear'] = test_date['regYear']
X_test['regMonth'] = test_date['regMonth']
X_test['creatMonth'] = test_date['creatMonth']
X_test['usedYear'] = test_date['usedDay'].apply(lambda x: x//356)
X_test['usedMonth'] = test_date['usedDay'].apply(lambda x: x//30)
X_test['usedWeek'] = test_date['usedDay'].apply(lambda x: x//7)

X_data.drop(['regDate', 'creatDate'], axis=1, inplace=True)
X_test.drop(['regDate', 'creatDate'], axis=1, inplace=True)

## 2.2 regionCode

In [6]:
X_data['city'] = X_data['regionCode'].astype('str').apply(lambda x: x[:2])
X_test['city'] = X_test['regionCode'].astype('str').apply(lambda x: x[:2])

## 2.2 分箱

## 2.2.1 power

In [7]:
power = X_data['power']
powerRange= []
for i, t in enumerate(power):
    powerRange.append(t // 30)
X_data['powerRange'] = pd.Series(powerRange)

power_test = X_test['power']
powerRange_test = []
for i, t in enumerate(power_test):
    powerRange_test.append(t//30)
X_test['powerRange'] = pd.Series(powerRange_test)

## 2.2.2 v系列

In [8]:
def cut_by_bins(bins, x):
    for i, b in enumerate(bins):
        if x >= b:
            return len(bins)-i-1
        
vs = ['v_{}'.format(i) for i in range(0, 15)]
vs_bins = []
for v in vs:
    v_data = X_data[v]
    v_test = X_test[v]
    bins = []
    for i in range(19, -1, -1):
        bins.append(X_data[v].quantile(0.05*i))
    X_data[v+'_bins'] = v_data.apply(lambda x: cut_by_bins(bins, x))
    X_test[v+'_bins'] = v_test.apply(lambda x: cut_by_bins(bins, x))
    vs_bins.append(v+'_bins')
vs_bins = ['v_{}_bins'.format(i) for i in [7, 4, 13, 14]]

In [9]:
X_data.head()

,SaleID,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,...,v_5_bins,v_6_bins,v_7_bins,v_8_bins,v_9_bins,v_10_bins,v_11_bins,v_12_bins,v_13_bins,v_14_bins
0,0,30.0,6,1.0,0.0,0.0,60,12.5,0.0,1046,...,2,14,15,2,16,6,18,3,14,16
1,1,40.0,1,2.0,0.0,0.0,0,15.0,NaN,4366,...,14,19,16,3,2,2,17,7,1,11
2,2,115.0,15,1.0,0.0,0.0,163,12.5,0.0,2806,...,7,18,18,11,3,2,17,14,6,6
3,3,109.0,10,0.0,0.0,1.0,193,15.0,0.0,434,...,18,17,14,4,0,3,15,8,0,4
4,4,110.0,5,1.0,0.0,0.0,68,5.0,0.0,6977,...,1,11,9,14,18,7,13,12,19,19


## 2.2.3 少数类合并

In [10]:
import seaborn as sns

sns.distplot(powerRange)

In [11]:
# 类别不平衡问题：model
def model_process(model):
    if model in {235.0, 246.0, 243.0, 244.0, 245.0, 209.0, 240.0, 242.0, 247.0}:
        model = 248.0
    return model
X_data['model'] = X_data['model'].apply(lambda x: model_process(x))
X_test['model'] = X_test['model'].apply(lambda x: model_process(x))

In [12]:
# usedMonth(删2)
def usedMonth_process(month):
    if month == 2:
        month = 3
    return month
X_data['usedMonth'] = X_data['usedMonth'].apply(lambda x: usedMonth_process(x))
X_test['usedMonth'] = X_test['usedMonth'].apply(lambda x: usedMonth_process(x))

In [13]:
# powerRange(改过大为20)
def powerRange_process(powerRange):
    if powerRange >= 20:
        powerRange = 20
    return powerRange
X_data['powerRange'] = X_data['powerRange'].apply(lambda x: powerRange_process(x))
X_test['powerRange'] = X_test['powerRange'].apply(lambda x: powerRange_process(x))

In [14]:
# 私用车和商务车分开
def bodyType_process(bodyType):
    if bodyType in [1.0, 7.0]: # 微型或搅拌
        return 0
    elif bodyType in [2.0, 3.0, 6.0]: # 商务车
        return 1
    else: # 豪华
        return 2
X_data['bodyTypeClass'] = X_data['bodyType'].apply(lambda x: bodyType_process(x))
X_test['bodyTypeClass'] = X_data['bodyType'].apply(lambda x: bodyType_process(x))
    
# 微型车单独分析
def bodyType1_process(bodyType):
    if bodyType != 1.0:
        bodyType = 0.0
    return bodyType
X_data['is_little_car'] = X_data['bodyType'].apply(lambda x: bodyType1_process(x))
X_test['is_little_car'] = X_test['bodyType'].apply(lambda x: bodyType1_process(x))

# 商务车单独分析
def bodyType6_process(bodyType):
    if bodyType != 6.0:
        bodyType = 0.0
    return bodyType
X_data['is_shangwu_car'] = X_data['bodyType'].apply(lambda x: bodyType6_process(x))
X_test['is_shangwu_car'] = X_test['bodyType'].apply(lambda x: bodyType6_process(x))

# 豪华轿车单独分析
def bodyType0_process(bodyType):
    if bodyType != 0.0:
        bodyType = 1.0
    return bodyType
X_data['is_haohua_car'] = X_data['bodyType'].apply(lambda x: bodyType6_process(x))
X_test['is_haohua_car'] = X_test['bodyType'].apply(lambda x: bodyType6_process(x))

# 柴油车单独分析
def fuelType1_process(fuelType):
    if fuelType != 1.0:
        fuelType = 0.0
    return fuelType
X_data['is_caiyou_car'] = X_data['fuelType'].apply(lambda x: fuelType1_process(x))
X_test['is_caiyou_car'] = X_test['fuelType'].apply(lambda x: fuelType1_process(x))

In [15]:
for key in ['model', 'brand', 'usedMonth', 'usedYear', 'creatMonth', 'regMonth', 'bodyType', 'kilometer', 'powerRange', 'fuelType', 'gearbox', 'notRepairedDamage', 'city'] + vs_bins:
    print(X_data[key].value_counts())

0.0      11762
19.0      9573
4.0       8445
1.0       6038
29.0      5186
48.0      5052
40.0      4502
26.0      4496
8.0       4391
31.0      3827
13.0      3762
17.0      3121
65.0      2730
49.0      2608
46.0      2454
30.0      2342
44.0      2195
5.0       2063
10.0      2004
21.0      1872
73.0      1789
11.0      1775
23.0      1696
22.0      1524
69.0      1522
63.0      1469
7.0       1460
16.0      1349
88.0      1309
66.0      1250
         ...  
193.0       48
190.0       47
214.0       47
68.0        45
238.0       44
189.0       43
18.0        42
89.0        38
141.0       37
133.0       35
216.0       30
248.0       30
202.0       28
151.0       26
226.0       26
234.0       23
231.0       23
233.0       20
198.0       18
224.0       18
227.0       17
237.0       17
230.0       16
220.0       16
239.0       14
223.0       13
236.0       11
232.0       10
229.0       10
241.0       10
Name: model, Length: 240, dtype: int64
0     31480
4     16737
14    16089
10    1424

# 2. 特征工程

## 2.1 count编码

In [16]:
# cnt_features = []
# for f in ['brand', 'model', 'usedYear', 'usedMonth', 'bodyType', 'kilometer', 'powerRange', 'notRepairedDamage', 'fuelType', 'gearbox'] + vs_bins:
#     X_data[f+'_cnt'] = X_data[f].map(X_data[f].value_counts())
#     X_test[f+'_cnt'] = X_test[f].map(X_data[f].value_counts())
#     cnt_features.append(f+'_cnt')

## 2.2 用数值特征对类别特征做统计刻画

In [17]:
# num_cate_features = []
# tar_features = ['v_{}'.format(i) for i in range(15)]
# for tar in tar_features:
#     for category in ['brand', 'model', 'usedYear', 'usedMonth']:
#         ori_group = X_data.groupby(by=[category])[tar]

#         cate_mean = ori_group.mean().reset_index().rename({tar: category+'_'+tar+'_mean'}, axis=1)
#         cate_std = ori_group.std().reset_index().rename({tar: category+'_'+tar+'_std'}, axis=1)
#         cate_max = ori_group.max().reset_index().rename({tar: category+'_'+tar+'_max'}, axis=1)
#         cate_min = ori_group.min().reset_index().rename({tar: category+'_'+tar+'_min'}, axis=1)
#         cate_median = ori_group.median().reset_index().rename({tar: category+'_'+tar+'_median'}, axis=1)
#     #     cate_quantile1 = ori_group.quantile(0.25).reset_index().rename({'price': category+'_q0.25'}, axis=1)
#     #     cate_quantile3 = ori_group.quantile(0.75).reset_index().rename({'price': category+'_q0.75'}, axis=1)

#         same_columns = [category]
#         X_data = X_data.merge(cate_mean, on=same_columns, how='left')
#         X_data = X_data.merge(cate_std, on=same_columns, how='left')
#         X_data = X_data.merge(cate_max, on=same_columns, how='left')
#         X_data = X_data.merge(cate_min, on=same_columns, how='left')
#         X_data = X_data.merge(cate_median, on=same_columns, how='left')
#     #     X_data = X_data.merge(cate_quantile1, on=same_columns, how='left')
#     #     X_data = X_data.merge(cate_quantile3, on=same_columns, how='left')
        
# #         num_cate_features.append(category+'_'+tar+'_mean')
# #         num_cate_features.append(category+'_'+tar+'_std')
# #         num_cate_features.append(category+'_'+tar+'_max')
# #         num_cate_features.append(category+'_'+tar+'_min')
#         num_cate_features.append(category+'_'+tar+'_median')


#         X_test = X_test.merge(cate_mean, on=same_columns, how='left')
#         X_test = X_test.merge(cate_std, on=same_columns, how='left')
#         X_test = X_test.merge(cate_max, on=same_columns, how='left')
#         X_test = X_test.merge(cate_min, on=same_columns, how='left')
#         X_test = X_test.merge(cate_median, on=same_columns, how='left')
#     #     X_test = X_test.merge(cate_quantile1, on=same_columns, how='left')
#     #     X_test = X_test.merge(cate_quantile3, on=same_columns, how='left')
    
# # X_data.drop(['timeRange'], axis=1, inplace=True)
# # X_test.drop(['timeRange'], axis=1, inplace=True)

## 2.3 类别特征的二阶交叉

In [18]:
two_cate_features = []

for f_pair in ([
    ['fuelType', 'gearbox'], ['brand', 'bodyType'], ['brand', 'notRepairedDamage'], ['model', 'notRepairedDamage'],
    ['model', 'bodyType'], ['bodyType', 'notRepairedDamage'], ['brand', 'model', 'notRepairedDamage'],
    ['brand', 'bodyType', 'notRepairedDamage'], ['fuelType', 'gearbox', 'notRepairedDamage'],
    ['brand', 'kilometer'], ['fuelType', 'powerRange'], ['powerRange', 'gearbox'],
    ['kilometer', 'notRepairedDamage'], ['is_little_car', 'brand'], ['model', 'powerRange'],
    ['is_little_car', 'model'], ['is_shangwu_car', 'brand'], ['regYear', 'kilometer'],
    ['is_shangwu_car', 'model'], ['model', 'kilometer'],
    ['is_caiyou_car', 'brand'], ['is_caiyou_car', 'model'], ['is_caiyou_car', 'powerRange']
]):
    pair_group = X_data.groupby(by=f_pair)['price']
    pair_mean = pair_group.mean().reset_index().rename({'price': '_'.join(f_pair)+'_mean'}, axis=1)
#     pair_std = pair_group.std().reset_index().rename({'price': '_'.join(f_pair)+'_std'}, axis=1)
#     pair_median = pair_group.median().reset_index().rename({'price': '_'.join(f_pair)+'_median'}, axis=1)
#     pair_max_min = (pair_group.max()-pair_group.min()).reset_index().rename({'price': '_'.join(f_pair)+'_max_min'}, axis=1)
#     pair_max = pair_group.max().reset_index().rename({'price': '_'.join(f_pair)+'_max'}, axis=1)
#     pair_min = pair_group.min().reset_index().rename({'price': '_'.join(f_pair)+'_min'}, axis=1)
#     pair_count = pair_group.count().reset_index().rename({'price': '_'.join(f_pair)+'_count'}, axis=1)
    
    two_cate_features.append('_'.join(f_pair)+'_mean')
#     two_cate_features.append('_'.join(f_pair)+'_std')
#     two_cate_features.append('_'.join(f_pair)+'_median')
#     two_cate_features.append('_'.join(f_pair)+'_max_min')
#     two_cate_features.append('_'.join(f_pair)+'_max')
#     two_cate_features.append('_'.join(f_pair)+'_min')
#     two_cate_features.append('_'.join(f_pair)+'_count')
    
    
    X_data = X_data.merge(pair_mean, on=f_pair, how='left')
#     X_data = X_data.merge(pair_std, on=f_pair, how='left')
#     X_data = X_data.merge(pair_median, on=f_pair, how='left')
#     X_data = X_data.merge(pair_max_min, on=f_pair, how='left')
#     X_data = X_data.merge(pair_max, on=f_pair, how='left')
#     X_data = X_data.merge(pair_min, on=f_pair, how='left')
#     X_data = X_data.merge(pair_count, on=f_pair, how='left')
    
    X_test = X_test.merge(pair_mean, on=f_pair, how='left')
#     X_test = X_test.merge(pair_std, on=f_pair, how='left')
#     X_test = X_test.merge(pair_median, on=f_pair, how='left')
#     X_test = X_test.merge(pair_max_min, on=f_pair, how='left')
#     X_test = X_test.merge(pair_max, on=f_pair, how='left')
#     X_test = X_test.merge(pair_min, on=f_pair, how='left')
#     X_test = X_test.merge(pair_count, on=f_pair, how='left')

## 2.4 target编码

In [19]:
# to_gen_features = ['brand', 'model', 'usedYear', 'usedMonth', 'bodyType', 'kilometer', 'powerRange', 'fuelType'] + vs_bins

# for category in to_gen_features:
#     ori_group = X_data.groupby(by=[category])['price']

# #     cate_mean = ori_group.mean().reset_index().rename({'price': category + '_mean'}, axis=1)
#     cate_std = ori_group.std().reset_index().rename({'price': category + '_std'}, axis=1)
# #     cate_max = ori_group.max().reset_index().rename({'price': category + '_max'}, axis=1)
# #     cate_min = ori_group.min().reset_index().rename({'price': category + '_min'}, axis=1)
#     cate_median = ori_group.median().reset_index().rename({'price': category+'_median'}, axis=1)
# #     cate_max_min = (ori_group.max()-ori_group.min()).reset_index().rename({'price': category + '_max_min'}, axis=1)
# #     cate_skew = ori_group.skew().reset_index().rename({'price': category+'_skew'}, axis=1)
# #     cate_quantile1 = ori_group.quantile(0.25).reset_index().rename({'price': category+'_q3'}, axis=1)
# #     cate_quantile3 = ori_group.quantile(0.75).reset_index().rename({'price': category+'_q3'}, axis=1)

#     same_columns = [category]
# #     X_data = X_data.merge(cate_mean, on=same_columns, how='left')
#     X_data = X_data.merge(cate_std, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_max, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_min, on=same_columns, how='left')
#     X_data = X_data.merge(cate_median, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_max_min, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_skew, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_quantile1, on=same_columns, how='left')
# #     X_data = X_data.merge(cate_quantile3, on=same_columns, how='left')

    
# #     X_test = X_test.merge(cate_mean, on=same_columns, how='left')
#     X_test = X_test.merge(cate_std, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_max, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_min, on=same_columns, how='left')
#     X_test = X_test.merge(cate_median, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_max_min, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_skew, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_quantile1, on=same_columns, how='left')
# #     X_test = X_test.merge(cate_quantile3, on=same_columns, how='left')
    
# # X_data.drop(['timeRange'], axis=1, inplace=True)
# # X_test.drop(['timeRange'], axis=1, inplace=True)
# target_features = []
# for cate in to_gen_features:
#     for s in ['median', 'std']:
#         feat = cate + '_' + s
#         target_features.append(feat)

In [20]:
def group_feature(df, key, target, aggs):   
    features = []
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{key}_{ag}'] = ag
        features.append(f'{key}_{ag}')
    print(agg_dict)
    t = df.groupby(key)[target].agg(agg_dict).reset_index() # groupby按key排列，agg
    return t, features

def extract_feature(df, train):
    tar_feats = []
    t, fs = group_feature(df, 'model','price',['mean'])
    train = pd.merge(train, t, on='model', how='left')
    tar_feats += fs
    
    t, fs = group_feature(df, 'brand','price',['mean'])
    train = pd.merge(train, t, on='brand', how='left')
    tar_feats += fs
    
    t, fs = group_feature(df, 'usedMonth','price',['mean'])
    train = pd.merge(train, t, on='usedMonth', how='left')
    tar_feats += fs
    
    for key in ['usedYear', 'creatMonth', 'regMonth', 'bodyType', 'kilometer', 'powerRange', 'fuelType', 'gearbox', 'notRepairedDamage'] + vs_bins:
        t, fs = group_feature(df, key,'price', ['mean'])
        train = pd.merge(train, t, on=key, how='left')
        tar_feats += fs
    
    
    return train, tar_feats

X_data, target_features = extract_feature(X_data, X_data)
X_test, _ = extract_feature(X_data, X_test)

{'model_mean': 'mean'}
{'brand_mean': 'mean'}
{'usedMonth_mean': 'mean'}
{'usedYear_mean': 'mean'}
{'creatMonth_mean': 'mean'}
{'regMonth_mean': 'mean'}
{'bodyType_mean': 'mean'}
{'kilometer_mean': 'mean'}
{'powerRange_mean': 'mean'}
{'fuelType_mean': 'mean'}
{'gearbox_mean': 'mean'}
{'notRepairedDamage_mean': 'mean'}
{'v_7_bins_mean': 'mean'}
{'v_4_bins_mean': 'mean'}
{'v_13_bins_mean': 'mean'}
{'v_14_bins_mean': 'mean'}
{'model_mean': 'mean'}
{'brand_mean': 'mean'}
{'usedMonth_mean': 'mean'}
{'usedYear_mean': 'mean'}
{'creatMonth_mean': 'mean'}
{'regMonth_mean': 'mean'}
{'bodyType_mean': 'mean'}
{'kilometer_mean': 'mean'}
{'powerRange_mean': 'mean'}
{'fuelType_mean': 'mean'}
{'gearbox_mean': 'mean'}
{'notRepairedDamage_mean': 'mean'}
{'v_7_bins_mean': 'mean'}
{'v_4_bins_mean': 'mean'}
{'v_13_bins_mean': 'mean'}
{'v_14_bins_mean': 'mean'}


## 2.5 v系列统计量

In [21]:
v_sat = ['v_sum', 'v_std', 'v_skew', 'v_median', 'v_max', 'v_min']

vs = ['v_{}'.format(i) for i in range(1, 15)]

X_data['v_sum'] = X_data[vs].apply(lambda x: x.sum(), axis=1)
# X_data['v_mean'] = X_data[vs].apply(lambda x: x.mean(), axis=1)
X_data['v_std'] = X_data[vs].apply(lambda x: x.std(), axis=1)
X_data['v_skew'] = X_data[vs].apply(lambda x: x.skew(), axis=1)
X_data['v_median'] = X_data[vs].apply(lambda x: x.median(), axis=1)
X_data['v_max'] = X_data[vs].apply(lambda x: x.max(), axis=1)
X_data['v_min'] = X_data[vs].apply(lambda x: x.min(), axis=1)
# X_data['v_max_min'] = X_data['v_max'] - X_data['v_min']

X_test['v_sum'] = X_test[vs].apply(lambda x: x.sum(), axis=1)
# X_test['v_mean'] = X_test[vs].apply(lambda x: x.mean(), axis=1)
X_test['v_std'] = X_test[vs].apply(lambda x: x.std(), axis=1)
X_test['v_skew'] = X_test[vs].apply(lambda x: x.skew(), axis=1)
X_test['v_median'] = X_test[vs].apply(lambda x: x.median(), axis=1)
X_test['v_max'] = X_test[vs].apply(lambda x: x.max(), axis=1)
X_test['v_min'] = X_test[vs].apply(lambda x: x.min(), axis=1)
# X_test['v_max_min'] = X_test['v_max'] - X_test['v_min']

# 3. 保存

In [22]:
baseline_features = ['power', 'kilometer', 'usedDay'] + ['v_{}'.format(i) for i in range(15)]

important_features = baseline_features + target_features + two_cate_features + v_sat #+ time_features + cate_features #+ num_cate_features# + used_per_features# +  +  + cnt_features + #+ usedPer_features# + two_cate_features
print(important_features)

['power', 'kilometer', 'usedDay', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'model_mean', 'brand_mean', 'usedMonth_mean', 'usedYear_mean', 'creatMonth_mean', 'regMonth_mean', 'bodyType_mean', 'kilometer_mean', 'powerRange_mean', 'fuelType_mean', 'gearbox_mean', 'notRepairedDamage_mean', 'v_7_bins_mean', 'v_4_bins_mean', 'v_13_bins_mean', 'v_14_bins_mean', 'fuelType_gearbox_mean', 'brand_bodyType_mean', 'brand_notRepairedDamage_mean', 'bodyType_notRepairedDamage_mean', 'brand_model_notRepairedDamage_mean', 'brand_bodyType_notRepairedDamage_mean', 'fuelType_gearbox_notRepairedDamage_mean', 'brand_kilometer_mean', 'fuelType_powerRange_mean', 'powerRange_gearbox_mean', 'kilometer_notRepairedDamage_mean', 'bodyType_kilometer_mean', 'powerRange_kilometer_mean', 'regYear_kilometer_mean', 'fuelType_kilometer_mean', 'bodyTypeClass_brand_mean', 'brand_powerRange_mean', 'is_caiyou_car_brand_mean', 'is_caiyou_car_powerRange_mean',

In [23]:
X_data = X_data[important_features + ['price']]
X_test = X_test[['SaleID'] + important_features]

In [24]:
print(X_data.shape)
print(X_test.shape)

(150000, 60)
(50000, 60)


In [25]:
X_test.head()

,SaleID,power,kilometer,usedDay,v_0,v_1,v_2,v_3,v_4,v_5,...,bodyTypeClass_brand_mean,brand_powerRange_mean,is_caiyou_car_brand_mean,is_caiyou_car_powerRange_mean,v_sum,v_std,v_skew,v_median,v_max,v_min
0,150000,313,15.0,1569,49.593127,5.246568,1.001130,-4.122264,0.737532,0.264405,...,4180.481366,24689.205797,11972.010623,26402.864341,-2.644593,3.198264,-0.379381,0.114179,5.246568,-7.050969
1,150001,75,12.5,6262,42.395926,-3.253950,-1.753754,3.646605,-0.725597,0.261745,...,8165.546053,3518.678879,4777.530577,2700.977119,-4.806143,2.100320,0.566978,-0.362799,3.679418,-3.796107
2,150002,109,7.0,2561,45.841370,4.704178,0.155391,-1.118443,-0.229160,0.260216,...,3545.582868,5861.628399,4777.530577,3264.498334,1.868203,1.958133,-0.416656,0.125154,4.704178,-4.926690
3,150003,160,7.0,2181,46.440649,4.319155,0.428897,-2.037916,-0.234757,0.260466,...,5637.839536,10955.511808,4059.459380,6259.852107,2.238006,2.001140,-0.660316,0.183596,4.319155,-4.864637
4,150004,75,15.0,6824,42.184604,-3.166234,-1.572058,2.604143,0.387498,0.250999,...,2177.195042,2125.500119,2574.672481,2700.977119,-1.688951,1.820168,0.178838,0.014300,3.616475,-3.197685


In [26]:
X_data.head()

,power,kilometer,usedDay,v_0,v_1,v_2,v_3,v_4,v_5,v_6,...,brand_powerRange_mean,is_caiyou_car_brand_mean,is_caiyou_car_powerRange_mean,v_sum,v_std,v_skew,v_median,v_max,v_min,price
0,60,12.5,4385,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,...,2241.242518,2887.676515,2700.977119,7.119150,1.790512,-0.097517,0.182612,3.966344,-2.881803,1850
1,0,15.0,4757,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,...,4611.427545,7543.488689,2211.556378,0.589442,2.217817,0.169456,0.128367,5.236112,-4.900482,3600
2,163,12.5,4382,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,...,9446.322115,9460.582761,6259.852107,2.228145,2.088335,-0.190080,0.088543,4.823792,-4.846749,6222
3,193,15.0,7125,45.687478,4.492574,-0.050616,0.883600,-2.228079,0.274293,0.110300,...,11404.367220,7371.301224,8071.047286,-3.005148,2.034433,0.115087,0.016697,4.492574,-4.509599,2400
4,68,5.0,1531,44.383511,2.031433,0.572169,-1.571239,2.246088,0.228036,0.073205,...,2014.171053,2888.178109,2700.977119,8.575578,1.353860,-0.228995,0.400102,2.834518,-1.896240,5200


In [27]:
X_data.to_hdf('Train_data.h5', 'df', mode='w', format='table')
X_test.to_hdf('Test_data.h5', 'df', mode='w', format='table')

In [28]:
X_data.columns

Index(['power', 'kilometer', 'usedDay', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13',
       'v_14', 'model_mean', 'brand_mean', 'usedMonth_mean', 'usedYear_mean',
       'creatMonth_mean', 'regMonth_mean', 'bodyType_mean', 'kilometer_mean',
       'powerRange_mean', 'fuelType_mean', 'gearbox_mean',
       'notRepairedDamage_mean', 'v_7_bins_mean', 'v_4_bins_mean',
       'v_13_bins_mean', 'v_14_bins_mean', 'fuelType_gearbox_mean',
       'brand_bodyType_mean', 'brand_notRepairedDamage_mean',
       'bodyType_notRepairedDamage_mean', 'brand_model_notRepairedDamage_mean',
       'brand_bodyType_notRepairedDamage_mean',
       'fuelType_gearbox_notRepairedDamage_mean', 'brand_kilometer_mean',
       'fuelType_powerRange_mean', 'powerRange_gearbox_mean',
       'kilometer_notRepairedDamage_mean', 'bodyType_kilometer_mean',
       'powerRange_kilometer_mean', 'regYear_kilometer_mean',
       'fuelType_kilometer_mean', 'bodyType

In [29]:
# X_data.to_hdf('Train_data_best.h5', 'df', mode='w', format='table')
# X_test.to_hdf('Test_data_best.h5', 'df', mode='w', format='table')

In [30]:
X_data.describe()

,power,kilometer,usedDay,v_0,v_1,v_2,v_3,v_4,v_5,v_6,...,brand_powerRange_mean,is_caiyou_car_brand_mean,is_caiyou_car_powerRange_mean,v_sum,v_std,v_skew,v_median,v_max,v_min,price
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,...,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,119.316547,12.597160,4540.297113,44.406268,-0.044809,0.080765,0.078833,0.017875,0.248204,0.044923,...,5923.327333,5923.327333,5923.327333,0.683094,1.901008,-0.094743,0.074050,3.930879,-3.706069,5923.327333
std,177.168419,3.919576,1968.065768,2.457548,3.641893,2.929618,2.026514,1.193661,0.045804,0.051743,...,5056.962829,3675.596347,4811.532390,8.835657,0.834086,0.463288,0.147619,2.435711,0.975005,7501.998477
min,0.000000,0.500000,88.000000,30.451976,-4.295589,-4.470671,-7.275037,-4.364565,0.000000,0.000000,...,95.000000,1021.039648,499.000000,-14.519591,0.217064,-2.464898,-0.717330,0.252238,-9.639552,11.000000
25%,75.000000,12.500000,3051.000000,43.135799,-3.192349,-0.970671,-1.462580,-0.921191,0.243615,0.000038,...,2750.405096,3288.485095,2700.977119,-3.405576,1.487325,-0.428436,0.025087,2.783740,-4.142137,1300.000000
50%,110.000000,15.000000,4567.000000,44.610266,-3.052671,-0.382947,0.099722,-0.075910,0.257798,0.000812,...,4611.427545,4889.144928,5295.686764,-0.636470,1.736188,-0.121213,0.069292,3.512605,-3.292269,3250.000000
75%,150.000000,15.000000,6025.000000,46.004721,4.000670,0.241335,1.565838,0.868758,0.265297,0.102009,...,8201.858431,7760.118483,7967.196806,2.180297,2.039011,0.172625,0.109791,4.348676,-3.141197,7700.000000
max,19312.000000,15.000000,9224.000000,52.304178,7.320308,19.035496,9.854702,6.829352,0.291838,0.151420,...,92950.500000,44155.777778,51269.357143,56.257998,7.314120,2.084582,2.718103,19.035496,0.000000,99999.000000
